In [2]:
# !pip install kydavra

# # !pip install boruta
# !pip install shap

In [4]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statistics
import random

# Feature Selection
# Filter
from sklearn.feature_selection import mutual_info_classif as MIC
from sklearn.feature_selection import VarianceThreshold
from kydavra import MultiSURFSelector
# Wrapper
import shap
from sklearn.feature_selection import RFE
# from boruta import BorutaPy

# model
import xgboost as xgb
from xgboost import XGBClassifier

# parameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# evaluate
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score


from sklearn.feature_selection import SelectKBest

# from functools import reduce

In [5]:
from __future__ import print_function, division
import numpy as np
import scipy as sp
from sklearn.utils import check_random_state, check_X_y
from sklearn.base import TransformerMixin, BaseEstimator
import warnings

class BorutaPy(BaseEstimator, TransformerMixin):
    """
    Improved Python implementation of the Boruta R package.
    The improvements of this implementation include:
    - Faster run times:
        Thanks to scikit-learn's fast implementation of the ensemble methods.
    - Scikit-learn like interface:
        Use BorutaPy just like any other scikit learner: fit, fit_transform and
        transform are all implemented in a similar fashion.
    - Modularity:
        Any ensemble method could be used: random forest, extra trees
        classifier, even gradient boosted trees.
    - Two step correction:
        The original Boruta code corrects for multiple testing in an overly
        conservative way. In this implementation, the Benjamini Hochberg FDR is
        used to correct in each iteration across active features. This means
        only those features are included in the correction which are still in
        the selection process. Following this, each that passed goes through a
        regular Bonferroni correction to check for the repeated testing over
        the iterations.
    - Percentile:
        Instead of using the max values of the shadow features the user can
        specify which percentile to use. This gives a finer control over this
        crucial parameter. For more info, please read about the perc parameter.
    - Automatic tree number:
        Setting the n_estimator to 'auto' will calculate the number of trees
        in each itartion based on the number of features under investigation.
        This way more trees are used when the training data has many feautres
        and less when most of the features have been rejected.
    - Ranking of features:
        After fitting BorutaPy it provides the user with ranking of features.
        Confirmed ones are 1, Tentatives are 2, and the rejected are ranked
        starting from 3, based on their feautre importance history through
        the iterations.
    We highly recommend using pruned trees with a depth between 3-7.
    For more, see the docs of these functions, and the examples below.
    Original code and method by: Miron B Kursa, https://m2.icm.edu.pl/boruta/
    Boruta is an all relevant feature selection method, while most other are
    minimal optimal; this means it tries to find all features carrying
    information usable for prediction, rather than finding a possibly compact
    subset of features on which some classifier has a minimal error.
    Why bother with all relevant feature selection?
    When you try to understand the phenomenon that made your data, you should
    care about all factors that contribute to it, not just the bluntest signs
    of it in context of your methodology (yes, minimal optimal set of features
    by definition depends on your classifier choice).
    Parameters
    ----------
    estimator : object
        A supervised learning estimator, with a 'fit' method that returns the
        feature_importances_ attribute. Important features must correspond to
        high absolute values in the feature_importances_.
    n_estimators : int or string, default = 1000
        If int sets the number of estimators in the chosen ensemble method.
        If 'auto' this is determined automatically based on the size of the
        dataset. The other parameters of the used estimators need to be set
        with initialisation.
    perc : int, default = 100
        Instead of the max we use the percentile defined by the user, to pick
        our threshold for comparison between shadow and real features. The max
        tend to be too stringent. This provides a finer control over this. The
        lower perc is the more false positives will be picked as relevant but
        also the less relevant features will be left out. The usual trade-off.
        The default is essentially the vanilla Boruta corresponding to the max.
    alpha : float, default = 0.05
        Level at which the corrected p-values will get rejected in both
        correction steps.
    two_step : Boolean, default = True
        If you want to use the original implementation of Boruta with Bonferroni
        correction only set this to False.
    max_iter : int, default = 100
        The number of maximum iterations to perform.
    random_state : int, RandomState instance or None; default=None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`.
    verbose : int, default=0
        Controls verbosity of output:
        - 0: no output
        - 1: displays iteration number
        - 2: which features have been selected already
    early_stopping : bool, default = False
        Whether to use early stopping to terminate the selection process
        before reaching `max_iter` iterations if the algorithm cannot
        confirm a tentative feature for `n_iter_no_change` iterations.
        Will speed up the process at a cost of a possibility of a
        worse result.
        
    n_iter_no_change : int, default = 20
        Ignored if `early_stopping` is False. The maximum amount of
        iterations without confirming a tentative feature. 
    Attributes
    ----------
    n_features_ : int
        The number of selected features.
    support_ : array of shape [n_features]
        The mask of selected features - only confirmed ones are True.
    support_weak_ : array of shape [n_features]
        The mask of selected tentative features, which haven't gained enough
        support during the max_iter number of iterations..
    ranking_ : array of shape [n_features]
        The feature ranking, such that ``ranking_[i]`` corresponds to the
        ranking position of the i-th feature. Selected (i.e., estimated
        best) features are assigned rank 1 and tentative features are assigned
        rank 2.
    importance_history_ : array-like, shape [n_features, n_iters]
        The calculated importance values for each feature across all iterations.  
    Examples
    --------
    
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from boruta import BorutaPy
    
    # load X and y
    # NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
    X = pd.read_csv('examples/test_X.csv', index_col=0).values
    y = pd.read_csv('examples/test_y.csv', header=None, index_col=0).values
    y = y.ravel()
    
    # define random forest classifier, with utilising all cores and
    # sampling in proportion to y labels
    rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
    
    # define Boruta feature selection method
    feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)
    
    # find all relevant features - 5 features should be selected
    feat_selector.fit(X, y)
    
    # check selected features - first 5 features are selected
    feat_selector.support_
    
    # check ranking of features
    feat_selector.ranking_
    
    # call transform() on X to filter it down to selected features
    X_filtered = feat_selector.transform(X)
    References
    ----------
    [1] Kursa M., Rudnicki W., "Feature Selection with the Boruta Package"
        Journal of Statistical Software, Vol. 36, Issue 11, Sep 2010
    """

    def __init__(self, estimator, n_estimators=1000, perc=100, alpha=0.05,
                 two_step=True, max_iter=100, random_state=None, verbose=0,
                 early_stopping=False, n_iter_no_change=20):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.perc = perc
        self.alpha = alpha
        self.two_step = two_step
        self.max_iter = max_iter
        self.random_state = random_state
        self.verbose = verbose
        self.early_stopping = early_stopping
        self.n_iter_no_change = n_iter_no_change
        self.__version__ = '0.3'
        self._is_xgboost = 'xgboost' in str(type(self.estimator))

    def fit(self, X, y):
        """
        Fits the Boruta feature selection with the provided estimator.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            The training input samples.
        y : array-like, shape = [n_samples]
            The target values.
        """

        return self._fit(X, y)

    def transform(self, X, weak=False, return_df=False):
        """
        Reduces the input X to the features selected by Boruta.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            The training input samples.
        weak: boolean, default = False
            If set to true, the tentative features are also used to reduce X.
        
        return_df : boolean, default = False
            If ``X`` if a pandas dataframe and this parameter is set to True,
            the transformed data will also be a dataframe.
        Returns
        -------
        X : array-like, shape = [n_samples, n_features_]
            The input matrix X's columns are reduced to the features which were
            selected by Boruta.
        """

        return self._transform(X, weak, return_df)

    def fit_transform(self, X, y, weak=False, return_df=False):
        """
        Fits Boruta, then reduces the input X to the selected features.
        Parameters
        ----------
        X : array-like, shape = [n_samples, n_features]
            The training input samples.
        y : array-like, shape = [n_samples]
            The target values.
        weak: boolean, default = False
            If set to true, the tentative features are also used to reduce X.
        return_df : boolean, default = False
            If ``X`` if a pandas dataframe and this parameter is set to True,
            the transformed data will also be a dataframe.
        Returns
        -------
        X : array-like, shape = [n_samples, n_features_]
            The input matrix X's columns are reduced to the features which were
            selected by Boruta.
        """

        self._fit(X, y)
        return self._transform(X, weak, return_df)

    def _validate_pandas_input(self, arg):
        try:
            return arg.values
        except AttributeError:
            raise ValueError(
                "input needs to be a numpy array or pandas data frame."
            )

    def _fit(self, X, y):
        # check input params
        self._check_params(X, y)

        if not isinstance(X, np.ndarray):
            X = self._validate_pandas_input(X) 
        if not isinstance(y, np.ndarray):
            y = self._validate_pandas_input(y)

        self.random_state = check_random_state(self.random_state)
        
        early_stopping = False
        if self.early_stopping:
            if self.n_iter_no_change >= self.max_iter:
                if self.verbose > 0:
                    print(
                        f"n_iter_no_change is bigger or equal to max_iter"
                        f"({self.n_iter_no_change} >= {self.max_iter}), "
                        f"early stopping will not be used."
                    )
            else:
                early_stopping = True
        
        # setup variables for Boruta
        n_sample, n_feat = X.shape
        print("setup variables for Boruta")
        print("n_sample",n_sample)
        print("n_feat",n_feat)
        _iter = 1
        # early stopping vars
        _same_iters = 1
        _last_dec_reg = None
        # holds the decision about each feature:
        # 0  - default state = tentative in original code
        # 1  - accepted in original code
        # -1 - rejected in original code
        dec_reg = np.zeros(n_feat, dtype=int)
        print("holds the decision about each feature 0 1 -1")
        print('dec_reg shape', dec_reg.shape)
        # counts how many times a given feature was more important than
        # the best of the shadow features
        hit_reg = np.zeros(n_feat, dtype=int)
        print("the best of the shadow features")
        print('hit_reg shape', hit_reg.shape)
        # these record the history of the iterations
        imp_history = np.zeros(n_feat, dtype=float)
        print(" these record the history of the iterations")
        print('imp_history shape', imp_history.shape)
        sha_max_history = []

        # set n_estimators
        if self.n_estimators != 'auto':
            self.estimator.set_params(n_estimators=self.n_estimators)

        # main feature selection loop
        while np.any(dec_reg == 0) and _iter < self.max_iter:
            # find optimal number of trees and depth
            if self.n_estimators == 'auto':
                # number of features that aren't rejected
                not_rejected = np.where(dec_reg >= 0)[0].shape[0]
                n_tree = self._get_tree_num(not_rejected)
                self.estimator.set_params(n_estimators=n_tree)

            # make sure we start with a new tree in each iteration
            if self._is_xgboost:
                self.estimator.set_params(random_state=self.random_state.randint(0, 10000))
            else:
                self.estimator.set_params(random_state=self.random_state)

            # add shadow attributes, shuffle them and train estimator, get imps
            cur_imp = self._add_shadows_get_imps(X, y, dec_reg)

            # get the threshold of shadow importances we will use for rejection
            imp_sha_max = np.percentile(cur_imp[1], self.perc)
            print('get the threshold of shadow importances we will use for rejection','imp_sha_max',imp_sha_max)

            # record importance history
            sha_max_history.append(imp_sha_max)
            imp_history = np.vstack((imp_history, cur_imp[0]))
        

            # register which feature is more imp than the max of shadows
            hit_reg = self._assign_hits(hit_reg, cur_imp, imp_sha_max)

            # based on hit_reg we check if a feature is doing better than
            # expected by chance
            dec_reg = self._do_tests(dec_reg, hit_reg, _iter)

            # print out confirmed features
            if self.verbose > 0 and _iter < self.max_iter:
                self._print_results(dec_reg, _iter, 0)
            if _iter < self.max_iter:
                _iter += 1
                
            # early stopping
            if early_stopping:
                if _last_dec_reg is not None and (_last_dec_reg == dec_reg).all():
                    _same_iters += 1
                    if self.verbose > 0:
                        print(
                            f"Early stopping: {_same_iters} out "
                            f"of {self.n_iter_no_change}"
                        )
                else:
                    _same_iters = 1
                    _last_dec_reg = dec_reg.copy()
                if _same_iters > self.n_iter_no_change:
                    break
        print('imp_history',imp_history)
        print('hit_reg',hit_reg)
        print('dec_reg',dec_reg)


        # we automatically apply R package's rough fix for tentative ones
        confirmed = np.where(dec_reg == 1)[0]
        tentative = np.where(dec_reg == 0)[0]
        # ignore the first row of zeros
        tentative_median = np.median(imp_history[1:, tentative], axis=0)
        # which tentative to keep
        tentative_confirmed = np.where(tentative_median
                                       > np.median(sha_max_history))[0]
        tentative = tentative[tentative_confirmed]

        print('confirmed',confirmed)
        print('tentative',tentative)
        print('tentative_median',tentative_median)
        print('tentative_confirmed',tentative_confirmed)

        # basic result variables
        self.n_features_ = confirmed.shape[0]
        self.support_ = np.zeros(n_feat, dtype=bool)
        self.support_[confirmed] = 1
        self.support_weak_ = np.zeros(n_feat, dtype=bool)
        self.support_weak_[tentative] = 1

        # ranking, confirmed variables are rank 1
        self.ranking_ = np.ones(n_feat, dtype=int)
        # tentative variables are rank 2
        self.ranking_[tentative] = 2
        # selected = confirmed and tentative
        selected = np.hstack((confirmed, tentative))
        # all rejected features are sorted by importance history
        not_selected = np.setdiff1d(np.arange(n_feat), selected)
        # large importance values should rank higher = lower ranks -> *(-1)
        imp_history_rejected = imp_history[1:, not_selected] * -1
        print('large importance values should rank higher = lower ranks -> *(-1)','imp_history_rejected',imp_history_rejected)


        # update rank for not_selected features
        if not_selected.shape[0] > 0:
                # calculate ranks in each iteration, then median of ranks across feats
                iter_ranks = self._nanrankdata(imp_history_rejected, axis=1)
                rank_medians = np.nanmedian(iter_ranks, axis=0)
                ranks = self._nanrankdata(rank_medians, axis=0)

                # set smallest rank to 3 if there are tentative feats
                if tentative.shape[0] > 0:
                    ranks = ranks - np.min(ranks) + 3
                else:
                    # and 2 otherwise
                    ranks = ranks - np.min(ranks) + 2
                self.ranking_[not_selected] = ranks
        else:
            # all are selected, thus we set feature supports to True
            self.support_ = np.ones(n_feat, dtype=bool)

        self.importance_history_ = imp_history

        # notify user
        if self.verbose > 0:
            self._print_results(dec_reg, _iter, 1)
        return self

    def _transform(self, X, weak=False, return_df=False):
        # sanity check
        try:
            self.ranking_
        except AttributeError:
            raise ValueError('You need to call the fit(X, y) method first.')

        if weak:
            indices = self.support_ + self.support_weak_
        else:
            indices = self.support_

        if return_df:
            X = X.iloc[:, indices]
        else:
            X = X[:, indices]
        return X

    def _get_tree_num(self, n_feat):
        depth = None
        try:
            depth = self.estimator.get_params()['max_depth']
        except KeyError:
            warnings.warn(
                "The estimator does not have a max_depth property, as a result "
                " the number of trees to use cannot be estimated automatically."
            )
        if depth == None:
            depth = 10
        # how many times a feature should be considered on average
        f_repr = 100
        # n_feat * 2 because the training matrix is extended with n shadow features
        multi = ((n_feat * 2) / (np.sqrt(n_feat * 2) * depth))
        n_estimators = int(multi * f_repr)
        return n_estimators

    def _get_imp(self, X, y):
        try:
            self.estimator.fit(X, y)
        except Exception as e:
            raise ValueError('Please check your X and y variable. The provided '
                             'estimator cannot be fitted to your data.\n' + str(e))
        try:
            imp = self.estimator.feature_importances_
        except Exception:
            raise ValueError('Only methods with feature_importance_ attribute '
                             'are currently supported in BorutaPy.')
        return imp

    def _get_shuffle(self, seq):
        self.random_state.shuffle(seq)
        return seq

    def _add_shadows_get_imps(self, X, y, dec_reg):
        # find features that are tentative still
        x_cur_ind = np.where(dec_reg >= 0)[0]
        x_cur = np.copy(X[:, x_cur_ind])
        x_cur_w = x_cur.shape[1]
        # deep copy the matrix for the shadow matrix
        x_sha = np.copy(x_cur)
        # make sure there's at least 5 columns in the shadow matrix for
        while (x_sha.shape[1] < 5):
            x_sha = np.hstack((x_sha, x_sha))
        # shuffle xSha
        x_sha = np.apply_along_axis(self._get_shuffle, 0, x_sha)
        # get importance of the merged matrix
        imp = self._get_imp(np.hstack((x_cur, x_sha)), y)
        # separate importances of real and shadow features
        imp_sha = imp[x_cur_w:]
        imp_real = np.zeros(X.shape[1])
        imp_real[:] = np.nan
        imp_real[x_cur_ind] = imp[:x_cur_w]
        return imp_real, imp_sha

    def _assign_hits(self, hit_reg, cur_imp, imp_sha_max):
        # register hits for features that did better than the best of shadows
        cur_imp_no_nan = cur_imp[0]
        cur_imp_no_nan[np.isnan(cur_imp_no_nan)] = 0
        hits = np.where(cur_imp_no_nan > imp_sha_max)[0]
        hit_reg[hits] += 1
        return hit_reg

    def _do_tests(self, dec_reg, hit_reg, _iter):
        active_features = np.where(dec_reg >= 0)[0]
        hits = hit_reg[active_features]
        # get uncorrected p values based on hit_reg
        to_accept_ps = sp.stats.binom.sf(hits - 1, _iter, .5).flatten()
        to_reject_ps = sp.stats.binom.cdf(hits, _iter, .5).flatten()

        if self.two_step:
            # two step multicor process
            # first we correct for testing several features in each round using FDR
            to_accept = self._fdrcorrection(to_accept_ps, alpha=self.alpha)[0]
            to_reject = self._fdrcorrection(to_reject_ps, alpha=self.alpha)[0]

            # second we correct for testing the same feature over and over again
            # using bonferroni
            to_accept2 = to_accept_ps <= self.alpha / float(_iter)
            to_reject2 = to_reject_ps <= self.alpha / float(_iter)

            # combine the two multi corrections, and get indexes
            to_accept *= to_accept2
            to_reject *= to_reject2
        else:
            # as in th original Boruta, we simply do bonferroni correction
            # with the total n_feat in each iteration
            to_accept = to_accept_ps <= self.alpha / float(len(dec_reg))
            to_reject = to_reject_ps <= self.alpha / float(len(dec_reg))

        # find features which are 0 and have been rejected or accepted
        to_accept = np.where((dec_reg[active_features] == 0) * to_accept)[0]
        to_reject = np.where((dec_reg[active_features] == 0) * to_reject)[0]

        # updating dec_reg
        dec_reg[active_features[to_accept]] = 1
        dec_reg[active_features[to_reject]] = -1
        return dec_reg

    def _fdrcorrection(self, pvals, alpha=0.05):
        """
        Benjamini/Hochberg p-value correction for false discovery rate, from
        statsmodels package. Included here for decoupling dependency on statsmodels.
        Parameters
        ----------
        pvals : array_like
            set of p-values of the individual tests.
        alpha : float
            error rate
        Returns
        -------
        rejected : array, bool
            True if a hypothesis is rejected, False if not
        pvalue-corrected : array
            pvalues adjusted for multiple hypothesis testing to limit FDR
        """
        pvals = np.asarray(pvals)
        pvals_sortind = np.argsort(pvals)
        pvals_sorted = np.take(pvals, pvals_sortind)
        nobs = len(pvals_sorted)
        ecdffactor = np.arange(1, nobs + 1) / float(nobs)

        reject = pvals_sorted <= ecdffactor * alpha
        if reject.any():
            rejectmax = max(np.nonzero(reject)[0])
            reject[:rejectmax] = True

        pvals_corrected_raw = pvals_sorted / ecdffactor
        pvals_corrected = np.minimum.accumulate(pvals_corrected_raw[::-1])[::-1]
        pvals_corrected[pvals_corrected > 1] = 1
        # reorder p-values and rejection mask to original order of pvals
        pvals_corrected_ = np.empty_like(pvals_corrected)
        pvals_corrected_[pvals_sortind] = pvals_corrected
        reject_ = np.empty_like(reject)
        reject_[pvals_sortind] = reject
        return reject_, pvals_corrected_

    def _nanrankdata(self, X, axis=1):
        """
        Replaces bottleneck's nanrankdata with scipy and numpy alternative.
        """
        ranks = sp.stats.mstats.rankdata(X, axis=axis)
        ranks[np.isnan(X)] = np.nan
        return ranks

    def _check_params(self, X, y):
        """
        Check hyperparameters as well as X and y before proceeding with fit.
        """
        # check X and y are consistent len, X is Array and y is column
        X, y = check_X_y(X, y)
        if self.perc <= 0 or self.perc > 100:
            raise ValueError('The percentile should be between 0 and 100.')

        if self.alpha <= 0 or self.alpha > 1:
            raise ValueError('Alpha should be between 0 and 1.')

    def _print_results(self, dec_reg, _iter, flag):
        n_iter = str(_iter) + ' / ' + str(self.max_iter)
        n_confirmed = np.where(dec_reg == 1)[0].shape[0]
        n_rejected = np.where(dec_reg == -1)[0].shape[0]
        cols = ['Iteration: ', 'Confirmed: ', 'Tentative: ', 'Rejected: ']

        # still in feature selection
        if flag == 0:
            n_tentative = np.where(dec_reg == 0)[0].shape[0]
            content = map(str, [n_iter, n_confirmed, n_tentative, n_rejected])
            if self.verbose == 1:
                output = cols[0] + n_iter
            elif self.verbose > 1:
                output = '\n'.join([x[0] + '\t' + x[1] for x in zip(cols, content)])

        # Boruta finished running and tentatives have been filtered
        else:
            n_tentative = np.sum(self.support_weak_)
            n_rejected = np.sum(~(self.support_|self.support_weak_))
            content = map(str, [n_iter, n_confirmed, n_tentative, n_rejected])
            result = '\n'.join([x[0] + '\t' + x[1] for x in zip(cols, content)])
            output = "\n\nBorutaPy finished running.\n\n" + result
        print(output)

In [6]:
'''
Created with love by Sigmoid
@Author - Basoc Nicoleta-Nina - nicoleta.basoc28@gmail.com
'''

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
# from .errors import MissingDataError, NonNumericDataError, NoSuchColumnError

class MultiSURFSelector:

    def __init__(self, n_neighbours: int = 7, n_features: int = 5) -> None:
        """
            Setting up the algorithm
        :param n_neighbors: int, default = 7
            The number of neighbors to consider when assigning feature
            importance scores. If a float number is provided, that percentage of
            training samples is used as the number of neighbors.
            More neighbors results in more accurate scores, but takes longer.
            
        :param n_features: int, default = 5
            The number of top features (according to the relieff score) to
            retain after feature selection is applied.
        """
        self.n_neighbours = n_neighbours
        self.n_features = n_features
        
    def define_tresh_multisurf(self, dataframe : pd.DataFrame, target : str) -> float:
        """
            Defining the treshold for the multiSURF algorithm
        :param dataframe: pandas DataFrame
             Data Frame on which the algorithm is applied
        :param target: str
             The column name of the value that we what to predict
        :return: float
            
        """
        X=dataframe.drop([target], axis=1).values
        distance_vector=[]

        # Defining the distance array based on which we will calculate the threshold.
        for i in range(len(X)):
            for j in range(len(X)):
                if i!=j:
                    distance_vector.append(np.linalg.norm(X[i]-X[j]))
        
        # Calculating the average distances and their stds and the value of the threshold.
        average_distance = np.array(distance_vector).mean()
        std_vector=np.std(distance_vector)/2
        diference_tresh_std=average_distance - std_vector
        return diference_tresh_std
        
    def select(self, dataframe : pd.DataFrame, target : str) -> list:
        """
            Selecting the most important columns
        :param dataframe: pandas DataFrame
             Data Frame on which the algorithm is applied
        :param target: str
             The column name of the value that we what to predict
        :return: list
            The list of features that our algorithm selects as the best ones
        """
        # Verifying if data frame has NaN-values.
        if dataframe.isna().values.any():
            raise MissingDataError("The passed data frame contains missing values")
            
        # if the given target doesn't match any column in the DataFrame raise an error
        if target not in dataframe:
            raise NoSuchColumnError(f'No such column {target}')
            
       # if a column has string values raise an error
        for col in dataframe:
            if not pd.api.types.is_numeric_dtype(dataframe[col]):
                raise NonNumericDataError('The given DataFrame contains non-numeric values')
            
        # Converting the data frame in numpy arrays.
        X = dataframe.drop([target], axis=1).values
        y = dataframe[target].values
        
        # Obtaining the number of features and samples.
        n_samples, feature_number = X.shape
        
        # Initialising the score of all features as 0.
        score = np.zeros(feature_number)
        
        # Using NearestNeighbors to define the nighbours for our target instance and all others.
        nbrs = NearestNeighbors(n_neighbors=n_samples,algorithm='kd_tree').fit(X)
        
        # Getting the distances of the neighbours and their indices.
        distances, indices = nbrs.kneighbors(X)
        
        # Getting how many classes we have to predict. 
        n_classes = np.unique(y).tolist()
        
        self.treshold=self.define_tresh_multisurf(dataframe, target)
        
        # Identifying hits and misses.
        for i in range(n_samples):
            
            # Initialising the miss and hit counters. 
            miss=dict()
            hit=[]
            
            # Getting all the classes of the target column.
            for j in n_classes:
                miss[j] = []
            
            # Finding misses and hits based on the threshold. 
            for n in range(1, self.n_neighbours):
                if np.linalg.norm(y[i] - y[indices[i][n]])<self.treshold:
                    if y[i] == y[indices[i][n]]:
                        hit.append(indices[i][n])
                    else:
                        miss[y[indices[i][n]]].append(indices[i][n])
            
            # Feature Weight update based on the right misses and hits.
            cont = dict()
            cont_diff_idx = float(len(y[y == y[i]])) / float(n_samples)
            
            # Computing the probabilities of every class.
            for label in n_classes:
                cont_diff_class = float(len(y[y == label])) / float(n_samples)
                cont[label] = cont_diff_class/ (1 - cont_diff_idx)
                
            # For hits.
            hit_term = np.zeros(feature_number)
            for h in hit:
                hit_term = np.array(hit_term) + np.array(abs(X[i, :] - X[h, :]))
                
            # For misses.
            miss_term = dict()
            for (label, miss_list) in miss.items():
                miss_term[label] = np.zeros(feature_number)
                for m in miss_list:
                    miss_term[label] = np.array(abs(X[i, :] - X[m, :])) + np.array(miss_term[label])
                
                # Update score for miss.
                score += miss_term[label] / (self.n_neighbours * n_samples)
                
            # Update score for hit.
            score -= hit_term / self.n_neighbours

        # Sorting the indexes based on the score.    
        result = np.argsort(score, 0)
        index = result[::-1][:self.n_features]
        
        # Return the columns based on their score.
        return list(dataframe.iloc[:, index].columns)                           


In [7]:
# for threshold
def filter_arr(sample_arr, filter_index):
    result_val = sample_arr[filter_index]
    result_index = filter_index
    compare_index = filter_index + 1
    if sample_arr[0] == sample_arr[filter_index]:
        return (0,0)
        
    if (len(sample_arr) -1 == filter_index):
        compare_index = filter_index - 1
    
    if (sample_arr[filter_index] == sample_arr[compare_index]):
        for index, val in enumerate(sample_arr):
            if (val == result_val):
                result_val = sample_arr[index - 1]
                result_index = index - 1

    return (result_val, result_index)



In [8]:
pd.set_option('display.max_columns', None)

In [9]:
df = pd.read_csv('pd_speech_features.csv', header=1)
df = df.drop(columns=['id'])
df

gender      PPE      DFA     RPDE  numPulses  numPeriodsPulses  \
0         1  0.85247  0.71826  0.57227        240               239   
1         1  0.76686  0.69481  0.53966        234               233   
2         1  0.85083  0.67604  0.58982        232               231   
3         0  0.41121  0.79672  0.59257        178               177   
4         0  0.32790  0.79782  0.53028        236               235   
..      ...      ...      ...      ...        ...               ...   
751       0  0.80903  0.56355  0.28385        417               416   
752       0  0.16084  0.56499  0.59194        415               413   
753       0  0.88389  0.72335  0.46815        381               380   
754       0  0.83782  0.74890  0.49823        340               339   
755       0  0.81304  0.76471  0.46374        340               339   

     meanPeriodPulses  stdDevPeriodPulses  locPctJitter  locAbsJitter  \
0            0.008064            0.000087       0.00218      0.000018   
1            0.008258            0.000073       0.00195      0.000016   
2            0.008340            0.000060       0.00176      0.000015   
3            0.010858            0.000183       0.00419      0.000046   
4            0.008162            0.002669       0.00535      0.000044   
..                ...                 ...           ...           ...   
751          0.004627            0.000052       0.00064      0.000003   
752          0.004550            0.000220       0.00143      0.000006   
753          0.005069            0.000103       0.00076      0.000004   
754          0.005679            0.000055       0.00092      0.000005   
755          0.005676            0.000037       0.00078      0.000004   

     rapJitter  ppq5Jitter  ddpJitter  locShimmer  locDbShimmer  apq3Shimmer  \
0      0.00067     0.00129    0.00200     0.05883         0.517      0.03011   
1      0.00052     0.00112    0.00157     0.05516         0.502      0.02320   
2      0.00057     0.00111    0.00171     0.09902         0.897      0.05094   
3      0.00149     0.00268    0.00446     0.05451         0.527      0.02395   
4      0.00166     0.00227    0.00499     0.05610         0.497      0.02909   
..         ...         ...        ...         ...           ...          ...   
751    0.00008     0.00022    0.00024     0.01947         0.171      0.01068   
752    0.00016     0.00041    0.00047     0.04291         0.482      0.02307   
753    0.00011     0.00030    0.00034     0.02978         0.263      0.01597   
754    0.00017     0.00041    0.00052     0.03552         0.311      0.01791   
755    0.00014     0.00033    0.00042     0.03702         0.326      0.01940   

     apq5Shimmer  apq11Shimmer  ddaShimmer  meanAutoCorrHarmonicity  \
0        0.03496       0.04828     0.09034                 0.970805   
1        0.03675       0.06195     0.06961                 0.984322   
2        0.06497       0.07772     0.15282                 0.974846   
3        0.02857       0.04462     0.07185                 0.968343   
4        0.03327       0.05278     0.08728                 0.975754   
..           ...           ...         ...                      ...   
751      0.01260       0.01547     0.03204                 0.997438   
752      0.02626       0.03272     0.06920                 0.976867   
753      0.01916       0.02271     0.04790                 0.995544   
754      0.02348       0.02648     0.05373                 0.988241   
755      0.02332       0.02765     0.05820                 0.996402   

     meanNoiseToHarmHarmonicity  meanHarmToNoiseHarmonicity  minIntensity  \
0                      0.036223                      18.995     69.997496   
1                      0.017974                      21.497     67.415903   
2                      0.026313                      17.651     62.661706   
3                      0.042003                      19.865     76.306989   
4                      0.027139                      19.557     76.645686   

In [10]:
col_names = list(df.columns[1:-1])
features = df[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
df1 = pd.DataFrame(features, columns = col_names)
df1['gender'] = df['gender']
df1['class']  = df['class']
# df1 = df1[ ['gender'] + [ col for col in df1.columns if col not in ['gender','class'] ] + ['class']]

df1

PPE       DFA      RPDE  numPulses  numPeriodsPulses  \
0    0.627644  0.256144  0.605835  -0.846892         -0.842373   
1    0.121620 -0.080433  0.368415  -0.907404         -0.902773   
2    0.617950 -0.349839  0.733609  -0.927575         -0.922907   
3   -1.980560  1.382279  0.753631  -1.472186         -1.466513   
4   -2.472989  1.398068  0.300123  -0.887233         -0.882640   
..        ...       ...       ...        ...               ...   
751  0.370878 -1.964406 -1.494032   0.938221          0.939445   
752 -3.460449 -1.943738  0.749044   0.918050          0.909245   
753  0.813362  0.329201 -0.152220   0.575147          0.577042   
754  0.541051  0.695920  0.066780   0.161646          0.164304   
755  0.394581  0.922840 -0.184327   0.161646          0.164304   

     meanPeriodPulses  stdDevPeriodPulses  locPctJitter  locAbsJitter  \
0            0.933328           -0.407251     -0.054993      0.037843   
1            1.040014           -0.426092     -0.142570     -0.027698   
2            1.084576           -0.443557     -0.214916     -0.088871   
3            2.464215           -0.275316      0.710353      1.256919   
4            0.987044            3.143597      1.152045      1.178269   
..                ...                 ...           ...           ...   
751         -0.949508           -0.454833     -0.641378     -0.601407   
752         -0.991826           -0.224083     -0.340570     -0.447165   
753         -0.707165           -0.385449     -0.595685     -0.562082   
754         -0.373097           -0.450845     -0.534762     -0.502220   
755         -0.374874           -0.475599     -0.588070     -0.538924   

     rapJitter  ppq5Jitter  ddpJitter  locShimmer  locDbShimmer  apq3Shimmer  \
0     0.066329    0.078201   0.062894   -0.201283     -0.233889    -0.190826   
1    -0.086722   -0.023255  -0.083360   -0.286750     -0.273123    -0.499987   
2    -0.035705   -0.029223  -0.035742    0.734660      0.760052     0.741128   
3     0.903010    0.907750   0.899601   -0.301887     -0.207732    -0.466431   
4     1.076468    0.663063   1.079867   -0.264860     -0.286201    -0.236462   
..         ...         ...        ...         ...           ...          ...   
751  -0.535673   -0.560374  -0.535725   -1.117898     -1.138898    -1.060144   
752  -0.454045   -0.446982  -0.457497   -0.572028     -0.325436    -0.505803   
753  -0.505063   -0.512630  -0.501713   -0.877799     -0.898260    -0.823464   
754  -0.443842   -0.446982  -0.440491   -0.744126     -0.772709    -0.736666   
755  -0.474452   -0.494726  -0.474503   -0.709194     -0.733475    -0.670002   

     apq5Shimmer  apq11Shimmer  ddaShimmer  meanAutoCorrHarmonicity  \
0      -0.228931     -0.208934   -0.190677                 0.145143   
1      -0.162993      0.194112   -0.499839                 0.356172   
2       0.876535      0.659075    0.741133                 0.208232   
3      -0.464316     -0.316846   -0.466432                 0.106706   
4      -0.291184     -0.076256   -0.236313                 0.222407   
..           ...           ...         ...                      ...   
751    -1.052596     -1.176304   -1.060147                 0.560941   
752    -0.549409     -0.667705   -0.505953                 0.239784   
753    -0.810948     -0.962840   -0.823616                 0.531372   
754    -0.651814     -0.851686   -0.736669                 0.417356   
755    -0.657708     -0.817189   -0.670004                 0.544767   

     meanNoiseToHarmHarmonicity  meanHarmToNoiseHarmonicity  minIntensity  \
0                     -0.142650                    0.024291     -0.093886   
1                     -0.317264                    0.473259     -0.385512   
2                     -0.237473                   -0.216881     -0.922563   
3                     -0.087344                    0.180407      0.618858   
4                     -0.229570                    0.125138      0.657118   
..                          ...                         ...     

In [11]:
# df1['class'] = df1.apply(lambda row: categorise(row), axis=1)
print('class 0 = ',df1['class'].value_counts()[0],'ตัว')
print('class 1 = ',df1['class'].value_counts()[1],'ตัว')
print('Majority Class= ', df1['class'].value_counts()[0]/len(df1))

class 0 =  192 ตัว
class 1 =  564 ตัว
Majority Class=  0.25396825396825395


In [12]:
# threshold:

log_2base = math.log(len(df1),2.0)
base2n = int(round(log_2base,0))
p2_5 = int(round(len(df1.columns)*0.025,0))
p5 =  int(round(len(df1.columns)*0.05,0))
p10 =  int(round(len(df1.columns)*0.1,0))
p20 =  int(round(len(df1.columns)*0.2,0))

print('treshold logbase2(n):', base2n)
print('treshold 2.5%:',p2_5)
print('treshold 5%:',p5)
print('treshold 10%:',p10)
print('treshold 20%:',p20)

thres_d = {'base2n':base2n,
              'p2_5':p2_5,
              'p5':p5,
              'p10':p10,
              'p20':p20}


thres_d


treshold logbase2(n): 10
treshold 2.5%: 19
treshold 5%: 38
treshold 10%: 75
treshold 20%: 151


{'base2n': 10, 'p2_5': 19, 'p5': 38, 'p10': 75, 'p20': 151}

In [13]:
X_ = df1.drop(['class'], axis=1)
len(X_.columns)

753

In [14]:
paramXGBoost_grid1 = {"subsample":[0.5, 0.75, 1],
              "colsample_bytree":[0.5, 0.75, 1],
              "max_depth":[5, 6, 7],
              "min_child_weight":[1,5,15],
              "learning_rate":[0.1, 0.01, 0.05],
              'eta': [0.01, 0.015, 0.02]
    }

In [15]:
paramXGBoost_grid2 = {"subsample":[0.2,0.6,0.9],
              "colsample_bytree":[0.5, 0.75, 1],
              "max_depth":[7,15,25],
              "min_child_weight":[0.1,0.5,1],
              "learning_rate":[0.1, 0.01, 0.05],
              'eta': [0.01, 0.015, 0.02]
    }

In [16]:
paramXGBoost_grid3 = {"subsample":[0.2,0.6,0.9],
              "colsample_bytree":[0.1,0.25, 0.5],
              "max_depth":[10,15,20],
              "min_child_weight":[20,40,60],
              "learning_rate":[0.1, 0.01, 0.05],
              'eta': [0.01, 0.015, 0.02]
    }

In [17]:
paramXGBoost_grid4 = {"subsample":[0.75,0.9,1],
              "colsample_bytree":[0.1,0.25, 0.5],
              "max_depth":[5,10,15],
              "min_child_weight":[5,10,20],
              "learning_rate":[0.1, 0.01, 0.05],
              'eta': [0.01, 0.015, 0.02]
    }

In [18]:
param_d = {'set1':paramXGBoost_grid1,
           'set2':paramXGBoost_grid2,
           'set3':paramXGBoost_grid3,
           'set4':paramXGBoost_grid4
          }

param_d

{'set1': {'subsample': [0.5, 0.75, 1],
  'colsample_bytree': [0.5, 0.75, 1],
  'max_depth': [5, 6, 7],
  'min_child_weight': [1, 5, 15],
  'learning_rate': [0.1, 0.01, 0.05],
  'eta': [0.01, 0.015, 0.02]},
 'set2': {'subsample': [0.2, 0.6, 0.9],
  'colsample_bytree': [0.5, 0.75, 1],
  'max_depth': [7, 15, 25],
  'min_child_weight': [0.1, 0.5, 1],
  'learning_rate': [0.1, 0.01, 0.05],
  'eta': [0.01, 0.015, 0.02]},
 'set3': {'subsample': [0.2, 0.6, 0.9],
  'colsample_bytree': [0.1, 0.25, 0.5],
  'max_depth': [10, 15, 20],
  'min_child_weight': [20, 40, 60],
  'learning_rate': [0.1, 0.01, 0.05],
  'eta': [0.01, 0.015, 0.02]},
 'set4': {'subsample': [0.75, 0.9, 1],
  'colsample_bytree': [0.1, 0.25, 0.5],
  'max_depth': [5, 10, 15],
  'min_child_weight': [5, 10, 20],
  'learning_rate': [0.1, 0.01, 0.05],
  'eta': [0.01, 0.015, 0.02]}}

# XGBoost

In [19]:
# baseline model

X = df1.drop(['class'], axis=1)
y = df1['class']
 


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123) 

""" do not need to split the data into training and test sets when using 
cross_val_score from the scikit-learn library. The purpose of 
cross_val_score is to perform cross-validation on the entire dataset, 
which automatically splits the data into training and validation sets 
for each fold. """
# XGBoost
xgbc = XGBClassifier(random_state=123)

## results F1 and stdF1 ##
avgF1_baselineXGBoost = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1_baselineXGBoost = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1_baselineXGBoost)
print('std F1 Score: ', stdF1_baselineXGBoost)



Mean F1 Score:  0.9058995249794035
std F1 Score:  0.029966930501710346


In [20]:
n_baselineXGBoost = len(X_.columns)
n_baselineXGBoost

753

In [21]:
# tuned baseline model_ parameterset1 


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgbc, param_grid = paramXGBoost_grid1,
                           cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
# Fit the grid search to the data
grid_search.fit(X,y)


print(grid_search.best_params_)


xgbc_tuned1 = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                   max_depth=grid_search.best_params_['max_depth'],
                                   colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                   min_child_weight=grid_search.best_params_['min_child_weight'],
                                   learning_rate=grid_search.best_params_['learning_rate'],
                                   eta=grid_search.best_params_['eta'])
xgbc_tuned1.fit(X,y)

print('\n')

## results F1 and stdF1 ##
avgF1set1_baselineXGBoost = cross_val_score(xgbc_tuned1, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1set1_baselineXGBoost = cross_val_score(xgbc_tuned1, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1set1_baselineXGBoost)
print('std F1 Score: ', stdF1set1_baselineXGBoost)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
{'colsample_bytree': 0.75, 'eta': 0.01, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'subsample': 0.5}


Mean F1 Score:  0.9053025519472531
std F1 Score:  0.029800940955119024


In [22]:
# tuned baseline model_ parameterset2


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgbc, param_grid = paramXGBoost_grid2,
                           cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
# Fit the grid search to the data
grid_search.fit(X,y)


print(grid_search.best_params_)


xgbc_tuned2 = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                   max_depth=grid_search.best_params_['max_depth'],
                                   colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                   min_child_weight=grid_search.best_params_['min_child_weight'],
                                   learning_rate=grid_search.best_params_['learning_rate'],
                                   eta=grid_search.best_params_['eta'])
xgbc_tuned2.fit(X,y)

print('\n')

## results F1 and stdF1 ##
avgF1set2_baselineXGBoost = cross_val_score(xgbc_tuned2, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1set2_baselineXGBoost = cross_val_score(xgbc_tuned2, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1set2_baselineXGBoost)
print('std F1 Score: ', stdF1set2_baselineXGBoost)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
{'colsample_bytree': 1, 'eta': 0.01, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 0.1, 'subsample': 0.6}


Mean F1 Score:  0.9114376885215977
std F1 Score:  0.03172384216880181


In [23]:
# tuned baseline model_ parameterset3


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgbc, param_grid = paramXGBoost_grid3,
                           cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
# Fit the grid search to the data
grid_search.fit(X,y)


print(grid_search.best_params_)


xgbc_tuned3 = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                   max_depth=grid_search.best_params_['max_depth'],
                                   colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                   min_child_weight=grid_search.best_params_['min_child_weight'],
                                   learning_rate=grid_search.best_params_['learning_rate'],
                                   eta=grid_search.best_params_['eta'])
xgbc_tuned3.fit(X,y)

print('\n')

## results F1 and stdF1 ##
avgF1set3_baselineXGBoost = cross_val_score(xgbc_tuned3, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1set3_baselineXGBoost = cross_val_score(xgbc_tuned3, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1set3_baselineXGBoost)
print('std F1 Score: ', stdF1set3_baselineXGBoost)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
{'colsample_bytree': 0.1, 'eta': 0.01, 'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 20, 'subsample': 0.9}


Mean F1 Score:  0.8937703976313915
std F1 Score:  0.04042853781335349


In [24]:
# tuned baseline model_ parameterset4


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = xgbc, param_grid = paramXGBoost_grid4,
                           cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
# Fit the grid search to the data
grid_search.fit(X,y)


print(grid_search.best_params_)


xgbc_tuned4 = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                   max_depth=grid_search.best_params_['max_depth'],
                                   colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                   min_child_weight=grid_search.best_params_['min_child_weight'],
                                   learning_rate=grid_search.best_params_['learning_rate'],
                                   eta=grid_search.best_params_['eta'])
xgbc_tuned4.fit(X,y)

## results F1 and stdF1 ##
avgF1set4_baselineXGBoost = cross_val_score(xgbc_tuned4, X, y, scoring="f1", cv = 10).mean() ##<3## 
stdF1set4_baselineXGBoost = cross_val_score(xgbc_tuned4, X, y, scoring="f1", cv = 10).std() ##<3##
print('Mean F1 Score: ', avgF1set4_baselineXGBoost)
print('std F1 Score: ', stdF1set4_baselineXGBoost)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
{'colsample_bytree': 0.1, 'eta': 0.01, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'subsample': 0.75}
Mean F1 Score:  0.9040658953937101
std F1 Score:  0.04253978660779925


In [25]:
output_basedlineXGBoost = {'n_baselineXGBoost':n_baselineXGBoost ,
                   'avgF1_baselineXGBoost':avgF1_baselineXGBoost ,
                   'stdF1_baselineXGBoost': stdF1_baselineXGBoost,
                   'avgF1set1_baselineXGBoost':avgF1set1_baselineXGBoost  ,
                   'stdF1set1_baselineXGBoost': stdF1set1_baselineXGBoost,
                   'avgF1set2_baselineXGBoost':avgF1set2_baselineXGBoost ,
                   'stdF1set2_baselineXGBoost':stdF1set2_baselineXGBoost ,
                   'avgF1set3_baselineXGBoost':avgF1set3_baselineXGBoost ,
                   'stdF1set3_baselineXGBoost':stdF1set3_baselineXGBoost ,
                   'avgF1set4_baselineXGBoost':avgF1set4_baselineXGBoost ,
                   'stdF1set4_baselineXGBoost':stdF1set4_baselineXGBoost ,
                     }
output_basedlineXGBoost

{'n_baselineXGBoost': 753,
 'avgF1_baselineXGBoost': 0.9058995249794035,
 'stdF1_baselineXGBoost': 0.029966930501710346,
 'avgF1set1_baselineXGBoost': 0.9053025519472531,
 'stdF1set1_baselineXGBoost': 0.029800940955119024,
 'avgF1set2_baselineXGBoost': 0.9114376885215977,
 'stdF1set2_baselineXGBoost': 0.03172384216880181,
 'avgF1set3_baselineXGBoost': 0.8937703976313915,
 'stdF1set3_baselineXGBoost': 0.04042853781335349,
 'avgF1set4_baselineXGBoost': 0.9040658953937101,
 'stdF1set4_baselineXGBoost': 0.04253978660779925}

In [26]:
output_basedlineXGBoost = pd.DataFrame([output_basedlineXGBoost])
output_basedlineXGBoost = output_basedlineXGBoost.transpose()
output_basedlineXGBoost.to_csv('output_basedlineXGBoost.csv', index=True)

## Feature Selection

# Filter

In [27]:
# Mutual Information: Filter method
# Mutual Information: Filter method

mi_XGBoostscore = MIC(X,y)
mi_XGBoostfeatures = pd.DataFrame({'Feature':list(X_.columns),
                            'value_MI':mi_XGBoostscore})
mi_XGBoostfeatures['value_MI'] = mi_XGBoostfeatures['value_MI'].map('{:,.19f}'.format)
""" the more Mutual Information is the more feature importance it will have"""
mi_XGBoostfeatures = mi_XGBoostfeatures.sort_values(by='value_MI',ascending=False)
# mi_XGBoostfeatures = mi_XGBoostfeatures.set_index('Feature')
mi_XGBoostfeatures = mi_XGBoostfeatures.reset_index()
mi_XGBoostfeatures.to_csv('mi_XGBoostfeatures.csv', index=True)

mi_XGBoostfeatures

,index,Feature,value_MI
0,426,tqwt_entropy_log_dec_35,0.1079123854058448018
1,124,std_delta_delta_log_energy,0.1064351759001511688
2,57,mean_MFCC_2nd_coef,0.0981874320149189739
3,133,std_8th_delta_delta,0.0962991901459435695
4,443,tqwt_TKEO_mean_dec_16,0.0958615775241309365
...,...,...,...
748,339,tqwt_energy_dec_20,0.0000000000000000000
749,342,tqwt_energy_dec_23,0.0000000000000000000
750,353,tqwt_energy_dec_34,0.0000000000000000000
751,355,tqwt_energy_dec_36,0.0000000000000000000


In [28]:
# variance treshold: Filter method
"""Variance threshold is a simple method for feature selection that removes all features 
whose variance doesn't meet a certain threshold."""

threshold = 0.1
selector = VarianceThreshold(threshold)
selector = selector.fit(X)
variances = selector.variances_

variancethres_XGBoostfeatures = pd.DataFrame({'Feature':list(X_.columns),
                            'value_variancethres':variances})
# variancethres_XGBoostfeatures['value_variancethres'] = variancethres_XGBoostfeatures['value_variancethres'].map('{:,.19f}'.format)


"""The idea behind this method is that features 
with low variance don't contain much information and can be removed without affecting 
the performance of the model."""
variancethres_XGBoostfeatures = variancethres_XGBoostfeatures.sort_values(by='value_variancethres',ascending=False)
# variancethres_XGBoostfeatures = variancethres_XGBoostfeatures.set_index('Feature')
variancethres_XGBoostfeatures = variancethres_XGBoostfeatures.reset_index()
variancethres_XGBoostfeatures.to_csv('variancethres_XGBoostfeatures.csv', index=True)

variancethres_XGBoostfeatures

,index,Feature,value_variancethres
0,492,tqwt_TKEO_std_dec_29,1.000000
1,181,det_TKEO_std_3_coef,1.000000
2,328,tqwt_energy_dec_9,1.000000
3,355,tqwt_energy_dec_36,1.000000
4,387,tqwt_entropy_shannon_dec_32,1.000000
...,...,...,...
748,522,tqwt_medianValue_dec_23,1.000000
749,229,Ea2,1.000000
750,466,tqwt_TKEO_std_dec_3,1.000000
751,495,tqwt_TKEO_std_dec_32,1.000000


In [29]:
# MultiSURF: Filter method

features = list(X_.columns)

msrf = MultiSURFSelector(n_features=len(df1.columns)-1)
selected_cols = msrf.select(df1,'class')

list_of_numbers = list(range(1, len(df1.columns)))

MultiSURF_XGBoostfeatures = pd.DataFrame({'Feature':selected_cols,
                            'value_MultiSURF':list_of_numbers})
# # MultiSURF_XGBoostfeatures['value_MultiSURF'] = MultiSURF_XGBoostfeatures['value_MultiSURF'].map('{:,.19f}'.format)
# เรียงจากเลขน้อยคือสำคัญสูงสุด
MultiSURF_XGBoostfeatures = MultiSURF_XGBoostfeatures.sort_values(by='value_MultiSURF',ascending=True)
# MultiSURF_XGBoostfeatures = MultiSURF_XGBoostfeatures.set_index('Feature')
MultiSURF_XGBoostfeatures = MultiSURF_XGBoostfeatures.reset_index()

MultiSURF_XGBoostfeatures.to_csv('MultiSURF_XGBoostfeatures.csv', index=True)
# 
MultiSURF_XGBoostfeatures

,index,Feature,value_MultiSURF
0,0,tqwt_TKEO_mean_dec_33,1
1,1,tqwt_TKEO_mean_dec_32,2
2,2,tqwt_energy_dec_33,3
3,3,tqwt_energy_dec_34,4
4,4,tqwt_TKEO_std_dec_33,5
...,...,...,...
748,748,mean_11th_delta,749
749,749,mean_10th_delta,750
750,750,mean_9th_delta_delta,751
751,751,mean_11th_delta_delta,752


# Wrapper

In [30]:
# SHAP: Wrapper method



# Explain model predictions using shap library:
explainer = shap.TreeExplainer(xgbc_tuned2) #### 
shap_values = explainer.shap_values(X)
vals= np.abs(shap_values).mean(0)

SHAP_XGBoostfeatures= pd.DataFrame({'Feature':list(X_.columns),
                                 'value_SHAP':vals})
SHAP_XGBoostfeatures['value_SHAP'] = SHAP_XGBoostfeatures['value_SHAP'].map('{:,.19f}'.format)
SHAP_XGBoostfeatures = SHAP_XGBoostfeatures.sort_values(by=['value_SHAP'],ascending=False)
SHAP_XGBoostfeatures = SHAP_XGBoostfeatures.set_index('Feature')
SHAP_XGBoostfeatures.to_csv('SHAP_XGBoostfeatures.csv', index=True)


SHAP_XGBoostfeatures

,value_SHAP
Feature,
std_delta_delta_log_energy,0.4071000218391418457
std_6th_delta_delta,0.2793149054050445557
std_7th_delta_delta,0.2317331433296203613
tqwt_entropy_log_dec_33,0.2028456181287765503
mean_MFCC_6th_coef,0.1673111021518707275
...,...
app_entropy_log_8_coef,0.0000000000000000000
app_entropy_log_9_coef,0.0000000000000000000
app_entropy_log_10_coef,0.0000000000000000000


In [31]:
# Recursive Feature Elimination(RFE): Wrapper method


"""In this example, the parameter "n_features_to_select" is set to 1, 
which means that only one feature will be selected at each iteration 
of RFE"""

rfe = RFE(estimator=xgbc_tuned2, n_features_to_select=1) #### changehere
rfe.fit(X,y)

rfe_XGBoostfeatures = pd.DataFrame({'Feature':list(X_.columns),
                            'value_rfe':rfe.ranking_})

# rfe_XGBoostfeatures['value_XGBooste'] = rfe_XGBoostfeatures['value_XGBooste'].astype(int)
# rfe_XGBoostfeatures['value_XGBooste'] = rfe_XGBoostfeatures['value_XGBooste'].map('{:,.19f}'.format)
"""The RFE assigns a ranking value to each feature, typically starting from 1 for 
the most important feature, and increasing for the less important features."""
rfe_XGBoostfeatures = rfe_XGBoostfeatures.sort_values(by=['value_rfe'],ascending=True)
rfe_XGBoostfeatures = rfe_XGBoostfeatures.set_index('Feature')

rfe_XGBoostfeatures.to_csv('rfe_XGBoostfeatures.csv', index=True)

rfe_XGBoostfeatures

,value_rfe
Feature,
std_delta_delta_log_energy,1
tqwt_stdValue_dec_12,2
tqwt_entropy_log_dec_33,3
tqwt_TKEO_std_dec_12,4
tqwt_entropy_log_dec_27,5
...,...
app_entropy_log_4_coef,749
app_LT_TKEO_std_10_coef,750
app_LT_TKEO_std_9_coef,751


In [32]:
# Boruta: Wrapper method: Wrapper method




feat_selector = BorutaPy(xgbc_tuned2, n_estimators='auto', verbose=2, random_state=1) #### changehere
# feat_selector.fit(X_train, y_train)
feat_selector = feat_selector.fit(np.array(X), np.array(y)) 
features = X.columns
feature_importance = feat_selector.ranking_
# feature_importance
boruta_XGBoostfeatures = pd.DataFrame(list(zip(X_.columns, feature_importance[0:].tolist())), 
                                  columns=['Feature','value_boruta'])

# output the feature importance ranking: 1 is most importance >ascending=True
boruta_XGBoostfeatures = boruta_XGBoostfeatures.sort_values(by=['value_boruta'],ascending=True)
boruta_XGBoostfeatures = boruta_XGBoostfeatures.set_index('Feature')

boruta_XGBoostfeatures.to_csv('boruta_XGBoostfeatures.csv', index=True)

boruta_XGBoostfeatures

setup variables for Boruta
n_sample 756
n_feat 753
holds the decision about each feature 0 1 -1
dec_reg shape (753,)
the best of the shadow features
hit_reg shape (753,)
 these record the history of the iterations
imp_history shape (753,)
get the threshold of shadow importances we will use for rejection imp_sha_max 0.010425981134176254
Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	753
Rejected: 	0
get the threshold of shadow importances we will use for rejection imp_sha_max 0.01055095437914133
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	753
Rejected: 	0
get the threshold of shadow importances we will use for rejection imp_sha_max 0.013083464466035366
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	753
Rejected: 	0
get the threshold of shadow importances we will use for rejection imp_sha_max 0.009226036258041859
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	753
Rejected: 	0
get the threshold of shadow importances we will use for rejection imp_sha_max 0.010417906567454338
Iteration: 	5 /

,value_boruta
Feature,
tqwt_TKEO_mean_dec_12,1
tqwt_maxValue_dec_19,1
tqwt_TKEO_mean_dec_19,1
tqwt_stdValue_dec_12,1
tqwt_TKEO_mean_dec_33,1
...,...
app_LT_entropy_log_9_coef,714
app_LT_entropy_log_10_coef,714
app_LT_TKEO_mean_6_coef,714


# single FS

# Filter

In [33]:
method_Filter = {'MI': mi_XGBoostfeatures,
          'variancethres': variancethres_XGBoostfeatures,
          'MultiSURF': MultiSURF_XGBoostfeatures
         }

In [34]:
output_filter = dict()


for tres,t in thres_d.items():
    for met,m in method_Filter.items():
        k =(filter_arr(m.filter(regex='value').squeeze(), t-1))   
        if k[1] != 0:
            n = k[1]+1 
            print("n"+"_single"+met+"_"+tres+"XGBoost",n)
            output_filter["n"+"_single"+met+"_"+tres+"XGBoost"] = n
            df_selected = m.iloc[:k[1]+1]
            df_filter = df1.filter(items=df_selected['Feature'])
            X = df_filter
            y = df1['class']
            
            ## before tuned ##
            xgbc = XGBClassifier(random_state=123)
            xgbc.fit(X,y)
            ## results F1 and stdF1 ##
            m = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).mean() 
            s = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).std() 
            print("avgF1"+"_single"+met+"_"+tres+"XGBoost" , m)
            print("stdF1"+"_single"+met+"_"+tres+"XGBoost",s)
            output_filter["avgF1"+"_single"+met+"_"+tres+"XGBoost"] = m
            output_filter["stdF1"+"_single"+met+"_"+tres+"XGBoost"] = s
            
            for parm, p in param_d.items():
                # Instantiate the grid search model
                grid_search = GridSearchCV(estimator = xgbc, param_grid = p,
                                        cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
                # Fit the grid search to the data
                grid_search.fit(X,y)
        #         print(grid_search.best_params_)
                xgbc_tuned = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                                max_depth=grid_search.best_params_['max_depth'],
                                                colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                                min_child_weight=grid_search.best_params_['min_child_weight'],
                                                learning_rate=grid_search.best_params_['learning_rate'],
                                                eta=grid_search.best_params_['eta'])
                xgbc_tuned.fit(X,y)

                ## results F1 and stdF1 ##
                m_tune  = cross_val_score(xgbc_tuned, X, y, scoring="f1", cv = 10).mean() 
                s_tune  = cross_val_score(xgbc_tuned, X, y, scoring="f1", cv = 10).std() 
                print("avgF1"+parm+"_single"+met+"_"+tres+"XGBoost" , m_tune)
                print("stdF1"+parm+"_single"+met+"_"+tres+"XGBoost",s_tune)
                output_filter["avgF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = m_tune
                output_filter["stdF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = s_tune

        else:
            output_filter["n"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            output_filter["avgF1"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            output_filter["stdF1"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            for parm, p in param_d.items():
                output_filter["avgF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = k[1]
                output_filter["stdF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = k[1]
         

        
        
 

n_singleMI_base2nXGBoost 10
avgF1_singleMI_base2nXGBoost 0.8841962758133347
stdF1_singleMI_base2nXGBoost 0.03809813456979118
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set1_singleMI_base2nXGBoost 0.8935726620741974
stdF1set1_singleMI_base2nXGBoost 0.023881968948570132
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set2_singleMI_base2nXGBoost 0.8951933770219889
stdF1set2_singleMI_base2nXGBoost 0.03171831993800357
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set3_singleMI_base2nXGBoost 0.8959336705565818
stdF1set3_singleMI_base2nXGBoost 0.027364140981115906
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set4_singleMI_base2nXGBoost 0.8979484614003376
stdF1set4_singleMI_base2nXGBoost 0.028058231001206812
n_singlevariancethres_base2nXGBoost 9
avgF1_singlevariancethres_base2nXGBoost 0.8723357830225794
stdF1_singlevariancethres_base2nXGBoost 0.03382144111024447
Fitting 5 folds for each of 729 candida

In [35]:
output_filter

{'n_singleMI_base2nXGBoost': 10,
 'avgF1_singleMI_base2nXGBoost': 0.8841962758133347,
 'stdF1_singleMI_base2nXGBoost': 0.03809813456979118,
 'avgF1set1_singleMI_base2nXGBoost': 0.8935726620741974,
 'stdF1set1_singleMI_base2nXGBoost': 0.023881968948570132,
 'avgF1set2_singleMI_base2nXGBoost': 0.8951933770219889,
 'stdF1set2_singleMI_base2nXGBoost': 0.03171831993800357,
 'avgF1set3_singleMI_base2nXGBoost': 0.8959336705565818,
 'stdF1set3_singleMI_base2nXGBoost': 0.027364140981115906,
 'avgF1set4_singleMI_base2nXGBoost': 0.8979484614003376,
 'stdF1set4_singleMI_base2nXGBoost': 0.028058231001206812,
 'n_singlevariancethres_base2nXGBoost': 9,
 'avgF1_singlevariancethres_base2nXGBoost': 0.8723357830225794,
 'stdF1_singlevariancethres_base2nXGBoost': 0.03382144111024447,
 'avgF1set1_singlevariancethres_base2nXGBoost': 0.8708495309352697,
 'stdF1set1_singlevariancethres_base2nXGBoost': 0.031303311585261666,
 'avgF1set2_singlevariancethres_base2nXGBoost': 0.8639422975229655,
 'stdF1set2_singlev

In [36]:

output_single_Filter_XGBoost = pd.DataFrame([output_filter])
output_single_Filter_XGBoost = output_single_Filter_XGBoost.transpose()
output_single_Filter_XGBoost.to_csv('output_single_Filter_XGBoost.csv', index=True)

# Wrapper

In [37]:
SHAP_XGBoostfeatures = pd.read_csv('SHAP_XGBoostfeatures.csv')
rfe_XGBoostfeatures = pd.read_csv('rfe_XGBoostfeatures.csv')
boruta_XGBoostfeatures = pd.read_csv('boruta_XGBoostfeatures.csv')

In [38]:
method_Wrapper = {  'SHAP':SHAP_XGBoostfeatures,
                    'rfe': rfe_XGBoostfeatures,
                    'boruta': boruta_XGBoostfeatures
                }

In [39]:
output_wrapper = dict()


for tres,t in thres_d.items():
    for met,m in method_Wrapper.items():
        k =(filter_arr(m.filter(regex='value').squeeze(), t-1))   
        if k[1] != 0:
            n = k[1]+1 
            print("n"+"_single"+met+"_"+tres+"XGBoost",n)
            output_wrapper["n"+"_single"+met+"_"+tres+"XGBoost"] = n
            df_selected = m.iloc[:k[1]+1]
            df_filter = df1.filter(items=df_selected['Feature'])
            X = df_filter
            y = df1['class']
            
            ## before tuned ##
            xgbc = XGBClassifier(random_state=123)
            xgbc.fit(X,y)
            ## results F1 and stdF1 ##
            m = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).mean() 
            s = cross_val_score(xgbc, X, y, scoring="f1", cv = 10).std() 
            print("avgF1"+"_single"+met+"_"+tres+"XGBoost" , m)
            print("stdF1"+"_single"+met+"_"+tres+"XGBoost",s)
            output_wrapper["avgF1"+"_single"+met+"_"+tres+"XGBoost"] = m
            output_wrapper["stdF1"+"_single"+met+"_"+tres+"XGBoost"] = s
            
            for parm, p in param_d.items():
                # Instantiate the grid search model
                grid_search = GridSearchCV(estimator = xgbc, param_grid = p,
                                        cv = 5, n_jobs = -1, verbose = 2,scoring='f1')
                # Fit the grid search to the data
                grid_search.fit(X,y)
        #         print(grid_search.best_params_)
                xgbc_tuned = XGBClassifier(subsample=grid_search.best_params_['subsample'],
                                                max_depth=grid_search.best_params_['max_depth'],
                                                colsample_bytree=grid_search.best_params_['colsample_bytree'],
                                                min_child_weight=grid_search.best_params_['min_child_weight'],
                                                learning_rate=grid_search.best_params_['learning_rate'],
                                                eta=grid_search.best_params_['eta'])
                xgbc_tuned.fit(X,y)

                ## results F1 and stdF1 ##
                m_tune  = cross_val_score(xgbc_tuned, X, y, scoring="f1", cv = 10).mean() 
                s_tune  = cross_val_score(xgbc_tuned, X, y, scoring="f1", cv = 10).std() 
                print("avgF1"+parm+"_single"+met+"_"+tres+"XGBoost" , m_tune)
                print("stdF1"+parm+"_single"+met+"_"+tres+"XGBoost",s_tune)
                output_wrapper["avgF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = m_tune
                output_wrapper["stdF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = s_tune

        else:
            output_wrapper["n"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            output_wrapper["avgF1"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            output_wrapper["stdF1"+"_single"+met+"_"+tres+"XGBoost"] = k[1]
            for parm, p in param_d.items():
                output_wrapper["avgF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = k[1]
                output_wrapper["stdF1"+parm+"_single"+met+"_"+tres+"XGBoost"] = k[1]
         

        
        
 

n_singleSHAP_base2nXGBoost 10
avgF1_singleSHAP_base2nXGBoost 0.9019864867882912
stdF1_singleSHAP_base2nXGBoost 0.03892192105643141
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set1_singleSHAP_base2nXGBoost 0.9050024021128905
stdF1set1_singleSHAP_base2nXGBoost 0.044773049618999694
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set2_singleSHAP_base2nXGBoost 0.9038314829650679
stdF1set2_singleSHAP_base2nXGBoost 0.03869828754078917
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set3_singleSHAP_base2nXGBoost 0.8870245446619673
stdF1set3_singleSHAP_base2nXGBoost 0.031062767770927823
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
avgF1set4_singleSHAP_base2nXGBoost 0.9013347662407423
stdF1set4_singleSHAP_base2nXGBoost 0.030108515331412317
n_singlerfe_base2nXGBoost 10
avgF1_singlerfe_base2nXGBoost 0.9120766518560043
stdF1_singlerfe_base2nXGBoost 0.01902561139943277
Fitting 5 folds for each of 729 candidates, to

In [40]:
output_wrapper

{'n_singleSHAP_base2nXGBoost': 10,
 'avgF1_singleSHAP_base2nXGBoost': 0.9019864867882912,
 'stdF1_singleSHAP_base2nXGBoost': 0.03892192105643141,
 'avgF1set1_singleSHAP_base2nXGBoost': 0.9050024021128905,
 'stdF1set1_singleSHAP_base2nXGBoost': 0.044773049618999694,
 'avgF1set2_singleSHAP_base2nXGBoost': 0.9038314829650679,
 'stdF1set2_singleSHAP_base2nXGBoost': 0.03869828754078917,
 'avgF1set3_singleSHAP_base2nXGBoost': 0.8870245446619673,
 'stdF1set3_singleSHAP_base2nXGBoost': 0.031062767770927823,
 'avgF1set4_singleSHAP_base2nXGBoost': 0.9013347662407423,
 'stdF1set4_singleSHAP_base2nXGBoost': 0.030108515331412317,
 'n_singlerfe_base2nXGBoost': 10,
 'avgF1_singlerfe_base2nXGBoost': 0.9120766518560043,
 'stdF1_singlerfe_base2nXGBoost': 0.01902561139943277,
 'avgF1set1_singlerfe_base2nXGBoost': 0.9059685550992755,
 'stdF1set1_singlerfe_base2nXGBoost': 0.035371317684861646,
 'avgF1set2_singlerfe_base2nXGBoost': 0.9031196104889764,
 'stdF1set2_singlerfe_base2nXGBoost': 0.0223521803775741

In [41]:

output_single_wrapper_XGBoost = pd.DataFrame([output_wrapper])
output_single_wrapper_XGBoost = output_single_wrapper_XGBoost.transpose()
output_single_wrapper_XGBoost.to_csv('output_single_wrapper_XGBoost.csv', index=True)